# Workout Data Workbook 
by Alex Guerrero

### Packages Imported

In [5]:
# from email import header
# import streamlit as st
import pandas as pd
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

### CSV File Import

In [6]:
filename = 'data/workout-data - main.csv'
workout_data = pd.read_csv(filename)

## Exploring of the data

In [7]:
# Printing the first 5 rows
workout_data.head()

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Total_volume
0,5/31/2021,Bodyweight,Weight,NaN,NaN,127.8,NaN,NaN,0.0
1,6/1/2021,Bodyweight,Weight,NaN,NaN,128.3,NaN,NaN,0.0
2,6/7/2021,Bodyweight,Weight,NaN,NaN,130.0,NaN,NaN,0.0
3,6/9/2021,Bodyweight,Weight,NaN,NaN,128.9,NaN,NaN,0.0
4,7/8/2021,Bodyweight,Weight,NaN,NaN,129.0,NaN,NaN,0.0


In [8]:
# Printing the last 5 rows
workout_data.tail()

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Total_volume
2712,3/28/2023,Leg Curl,Glutes_Hamstrings,4.0,10.0,20.0,NaN,one leg,200.0
2713,3/28/2023,Machine Calf Raise,Calves,1.0,15.0,30.0,NaN,NaN,450.0
2714,3/28/2023,Machine Calf Raise,Calves,2.0,10.0,40.0,NaN,NaN,400.0
2715,3/28/2023,Machine Calf Raise,Calves,3.0,10.0,50.0,NaN,NaN,500.0
2716,3/28/2023,Machine Calf Raise,Calves,4.0,10.0,40.0,NaN,NaN,400.0


In [9]:
# date shape
workout_data.shape

(2717, 9)

In [10]:
# printing data columns
workout_data.columns

Index(['Date', 'Exercise', 'Muscle_group', 'Sets', 'Reps', 'Weight',
       'Duration', 'Note', 'Total_volume'],
      dtype='object')

In [11]:
# Dara more information
workout_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2717 entries, 0 to 2716
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          2717 non-null   object 
 1   Exercise      2717 non-null   object 
 2   Muscle_group  2717 non-null   object 
 3   Sets          2629 non-null   float64
 4   Reps          2603 non-null   float64
 5   Weight        1945 non-null   float64
 6   Duration      1 non-null      float64
 7   Note          236 non-null    object 
 8   Total_volume  2717 non-null   float64
dtypes: float64(5), object(4)
memory usage: 191.2+ KB


In [12]:
# checking and removing NaN rows
workout_data[workout_data.Date.isnull()]

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Total_volume


### Dataframe Clean Up

In [13]:
# columns clean up
data_columns = ['Date', 'Exercise', 'Muscle_group', 'Sets', 'Reps', 'Weight', 'Duration', 'Note', 'Total_volume']
workout_data = workout_data[data_columns]

# rows clean up - removing NaN rows
workout_data = workout_data[workout_data.Date.notnull()]

# changing date field to datetime
workout_data['Date'] = pd.to_datetime(workout_data['Date'], format="%m/%d/%Y")

# sorting dataframe according to date, exercise, sets
workout_data = workout_data.sort_values(by=['Date','Exercise','Sets'])

In [14]:
# re-checking dataframe shape
workout_data.shape

(2717, 9)

### Calculations

In [15]:
# Date Parameters -- Calculations
today = date.today()
start_of_week = today - relativedelta(days=today.weekday())
end_of_Week = start_of_week + relativedelta(days=6)
first_day_of_month = today.replace(day=1)
next_month_first_day = today + relativedelta(months=1, day=1)

# Formatting date parameters into string
today = today.strftime('%m-%d-%Y')
start_of_week = start_of_week.strftime('%m-%d-%Y')
end_of_Week = end_of_Week.strftime('%m-%d-%Y')
first_day_of_month = first_day_of_month.strftime('%m-%d-%Y')
next_month_first_day = next_month_first_day.strftime('%m-%d-%Y')

## KPIs

In [16]:
# total count of record in current week
weekly_count = len(workout_data.query(" Date >= @start_of_week & Date <= @end_of_Week")['Date'].unique())
print(f"Weekly Record Count: {weekly_count}")

Weekly Record Count: 0


In [17]:
# total count of record in current month
monthly_count = len(workout_data.query(" Date >= @first_day_of_month & Date < @next_month_first_day")['Date'].unique())
print(f"Montly Record Count: {monthly_count}")

Montly Record Count: 0


In [18]:
# unique date total record count
total_records_count = len(workout_data.Date.unique())
print(f"Total record count: {total_records_count}")

Total record count: 203


In [19]:
# total volume per day
total_volume_per_day = workout_data[["Date","Total_volume"]].groupby(by="Date").sum("Total_volume")
total_volume_per_day.reset_index()

,Date,Total_volume
0,2021-05-31,0.0
1,2021-06-01,0.0
2,2021-06-07,0.0
3,2021-06-09,0.0
4,2021-07-08,0.0
...,...,...
198,2023-03-24,0.0
199,2023-03-25,0.0
200,2023-03-26,9720.0
201,2023-03-27,10770.0


In [20]:
# bodyweight analysis
BODYWEIGHT_EXERCISES = ["Pull Up", "Chin Up", "Dips", "Push Up", "Squat"]
columns_name = ["Exercise", "Min", "Max", "Avg"]
df = []

for exercise in BODYWEIGHT_EXERCISES:
    min = round(workout_data[workout_data["Exercise"] == exercise].min()["Reps"])
    max = round(workout_data[workout_data["Exercise"] == exercise].max()["Reps"])
    avg = round(workout_data[workout_data["Exercise"] == exercise].mean(numeric_only=True)["Reps"])
    
    entry = [exercise, min, max, avg]
    df.append(entry)
    
df = pd.DataFrame(df, columns=columns_name)
df

,Exercise,Min,Max,Avg
0,Pull Up,2,10,5
1,Chin Up,3,10,6
2,Dips,3,20,10
3,Push Up,10,24,13
4,Squat,15,25,21


### Latest Record

In [21]:
latest_record = workout_data[workout_data['Date'] == workout_data['Date'].unique()[-1]]
latest_record

,Date,Exercise,Muscle_group,Sets,Reps,Weight,Duration,Note,Total_volume
2709,2023-03-28,Leg Curl,Glutes_Hamstrings,1.0,10.0,20.0,NaN,both legs,200.0
2710,2023-03-28,Leg Curl,Glutes_Hamstrings,2.0,10.0,30.0,NaN,both legs,300.0
2711,2023-03-28,Leg Curl,Glutes_Hamstrings,3.0,10.0,20.0,NaN,one leg,200.0
2712,2023-03-28,Leg Curl,Glutes_Hamstrings,4.0,10.0,20.0,NaN,one leg,200.0
2705,2023-03-28,Leg Extension,Quadriceps,1.0,15.0,50.0,NaN,both legs,750.0
2706,2023-03-28,Leg Extension,Quadriceps,2.0,15.0,30.0,NaN,one leg,450.0
2707,2023-03-28,Leg Extension,Quadriceps,3.0,15.0,20.0,NaN,one leg,300.0
2708,2023-03-28,Leg Extension,Quadriceps,4.0,10.0,20.0,NaN,one leg,200.0
2701,2023-03-28,Leg Press,Quadriceps,1.0,15.0,60.0,NaN,NaN,900.0
2702,2023-03-28,Leg Press,Quadriceps,2.0,15.0,80.0,NaN,NaN,1200.0


In [22]:
print(f"Recent muscle groups worked out: {workout_data[workout_data['Date'] == workout_data['Date'].unique()[-1]]['Muscle_group'].unique()}")


Recent muscle groups worked out: ['Glutes_Hamstrings' 'Quadriceps' 'Calves']


In [23]:
# what muscle did we work out last
for unique_muscle in latest_record['Muscle_group'].unique():
    print(unique_muscle)

Glutes_Hamstrings
Quadriceps
Calves


In [28]:
workout_data['Exercise'].unique()

array(['Bodyweight', 'Dumbbell Bench Press',
       'Dumbbell Incline Bench Press',
       'Dumbbell Overhead Triceps Extension', 'Dumbbell Shoulder Press',
       'Front Raise', 'Lat Raise', 'Chin Up', 'Dips', 'Pull Up',
       'Dumbbell Shrug', 'Note', 'Parallette Push-Up', 'Lat Pull-Down',
       'Reverse-Grip Lat Pull-Down', 'Seated Cable Row (V Handle)',
       'Machine Row', 'Ez Barbell Bicep Curl', 'Hammer Curl', 'Leg Curl',
       'Leg extension', 'Machine Calf Raise', 'Machine Seated Leg Press',
       'Bicep Curl', 'Military (Bbl) Shoulder Press',
       'Single Arm Tricep Pushdown / Extension', 'Dumbbell Row',
       'Leg Extension', 'Machine Rear Delt (Reverse) Fly',
       'MTS Front Pull-Down', 'MTS Row', 'Ab Crunch', 'Flutter Kicks',
       'Barbell T-Bar Row', 'Plank', 'Sit-Up', 'Underhand Barbell Row',
       'Smith Machine Squat', 'Barbell Squat',
       'Dumbbell Bulgarian Split-Squat',
       'Cable Overhead Triceps Extension', 'Machine Butterfly',
       'Machine I